## Obliczanie miar dla wyników detekcji systemu

W tym notebooku zarówno obliczane były finalne wyniki WER, CER oraz MER, ale i tutaj zapisane były przykładowe detekcje do pokazania w analizie wyników w pracy magisterskiej. 

Wyniki uzyskane z dwóch ostatnich komórek tego notebooka zostały zaprezentowane w analizie wyników pracy magisterskiej. 

Część tego notebooka (zaznaczona odpowiednio) była przeliczona na środowisku Google Colabolatory ze względu na inną architekturę mojej lokalnej maszyny w porównaniu do środowiska w którym uczony był model. Niemniej jednak finalne obliczenia na zapisanych detekcjach zostały przeprowadzone lokalnie. 

In [4]:
# importy oraz przygotowanie pomocnych funkcji/narzędzi

import jiwer
import pytesseract
import cv2
import os 
import numpy as np
import pickle
import io
import torch
import pandas as pd

# funkcje do  preprocessingu obrazów przed detekcjami tesseract
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

def thresholding(image):
    return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

# transformacje potrzebne do poprawnego obliczania WER/MER/CER 
transforms = jiwer.Compose(
    [
        jiwer.RemoveEmptyStrings(),
        jiwer.ToLowerCase(),
        jiwer.RemoveMultipleSpaces(),
        jiwer.Strip(),
        jiwer.RemovePunctuation(),
        jiwer.ReduceToListOfListOfWords(),
    ]
)

**Przeliczone na collabolatory:** komórka bezpośrednio pod spodem została przeliczona w tym samym środowisku co trenowanie modelu. Po obliczeniach wyniki zostały zapisane z pomocą biblioteki `pickle` a następnie przeliczone lokalnie (kolejne komórki).

In [ ]:
# całość poniej była puszczona na colabie, ze względu na błędy modelu kiedy puszczałem go lokalnie
# tak pozyskałem wyniki (results) dla zbiorów treningowego, validacyjnego oraz testowego
# te wyniki uzywam ponizej razem z biblioteką jitter do obliczenia wynikow WER

######################
# from ultralytics import YOLO


# print("loading model")
# model = YOLO("ran_on_colab/weights/best_best.pt")
# print("loaded model")

# path_to_train_im = "ran_on_colab/images/test/"
# path_to_train_info_excel = "data/generated_labels/first_gen_2024-06-24.xlsx"

# files_np = np.array(os.listdir(path_to_train_im))
# file_paths = np.char.add(path_to_train_im, files_np)

# file_paths_short = file_paths[:20]

# results = model(source=file_paths_short.tolist())
######################

Ładowanie plików pickle zwróconych przez bibliotekę torch musi zostać przeprowadzone w specyficzny sposób ze wzgledu na inne architektury sprzętu na którym prowadzona była nauka modelu (GPU) a na którym liczone są miary (CPU).


In [5]:
# ze wzgledu na roznice maszyn potrzeba otworzyć w specjlany sposob:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

with open("ran_on_colab/results_saved/train/train_results.pkl", "rb") as file:
    loaded_train_results = CPU_Unpickler(file).load()

with open("ran_on_colab/results_saved/test/test_results.pkl", "rb") as file:
    loaded_test_results = CPU_Unpickler(file).load()
    
with open("ran_on_colab/results_saved/val/val_results.pkl", "rb") as file:
    loaded_val_results = CPU_Unpickler(file).load()

In [6]:
# ładowanie zawartości wygenerowanych etykiet z pliku utworzonego podczas generacji
# detekcja będzie porównywana do generacji, na tej podstawie zostaną obliczone miary detekcji

path_to_train_info_excel = "data/generated_labels/first_gen_2024-06-24.xlsx"
generated_df = pd.read_excel(path_to_train_info_excel)

generated_df.head()

,Unnamed: 0,label_tag,label_type,uwagi,odbiorca,nadawca
0,0,0,dhl_1,Proszę zadzwonić domofonem.,Magdalena Kozak ul. Michałowska 113 03-767 W...,Mateusz Stępień ul. Nawigacyjna 19 60-480 Po...
1,0,1,inpost_3,"Upewnić się, że ktoś odbierze",MORSKI KWIAT ul. Wałuszewska 55 03-005 Warsz...,SŁONECZNA POLANA ul. Obrzańska 86 60-114 Poz...
2,0,2,inpost_3,Zadzwonić na numer kontaktowy,Gabriela Pawlik ul. Mickiewicza Adama 113 81...,Maria Nowak ul. Generała Fiszera Józefa 20 8...
3,0,3,dhl_1,Proszę o delikatne obchodzeni,Mariusz Piekarski ul. Mała 144 15-317 Białys...,Krzysztof Kwiatkowski ul. Brzozowa 125 43-60...
4,0,4,pocztex_1,Proszę dostarczyć na tylną fu,Paulina Łukasik ul. Wierzbowa 64 41-100 Siem...,Joanna Borowska ul. Płońska 48 03-683 Warsza...


### Wyliczenia miar

W następnych dwóch komórkach obliczane są miary dla wyników detekcji na zbiorze treningowym oraz walidacyjnym (odpowiednio opisane komentarzem). Dodatkowo w trakcie obliczeń zapisywane są przykładowe detekcje w celach pokazowych (obecnie zakomentowana linijka). Finalnie przez odkomentowanie odpowiedniej linijki (opisana komentarzem) da się spojrzeć na pojedyńcze detekcje w porównaniu z targetem (wygenerowanymi danymi).

In [12]:
# obliczenia dla zbioru treningowego
from pytesseract import Output

class_names_dict = {0:"nadawca",
                    1:"odbiorca",
                    2:"uwagi"}

nadawca_scores = []
odbiorca_scores = []
uwagi_scores = []

nadawca_scores_cer = []
odbiorca_scores_cer = []
uwagi_scores_cer = []

nadawca_scores_mer = []
odbiorca_scores_mer = []
uwagi_scores_mer = []

inpost_wer = []
dhl_wer = []
dpd_wer = []
pocztex_wer = []


for k, result in enumerate(loaded_train_results):
    orig_image = result.orig_img
    fullpath = result.path
    number = int(fullpath.split("_")[-1][:-5]) # zadziała dla train i val, nie dla test
    detected_classes = result.boxes.cls
    detected_boxes = result.boxes
    
    for i, class_detected in enumerate(detected_classes):
        class_int = int(class_detected.numpy())
        [x1, y1, x2, y2] = detected_boxes[i].xyxy.numpy().astype(int)[0]
        fragment = orig_image[y1:y2, x1:x2, :] # działa! 
        
        # #blur
        # img = cv2.blur(fragment, (10,10))
        
        # #sharpen
        # kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
        # img = cv2.filter2D(img, -1, kernel)
        
        img = get_grayscale(fragment)
        img = cv2.GaussianBlur(img,(5,5),0)
        # img = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\
        #     cv2.THRESH_BINARY,11,2)
        img = thresholding(img)
        
        #open
        kernel = np.ones((4,4),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        
        #close
        kernel = np.ones((4,4),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        
        
        text_on_fragment = pytesseract.image_to_string(img)

        text_category = class_names_dict[class_int]
        text_to_decipher = generated_df.iloc[number][text_category]
        label_type = generated_df.iloc[number]['label_type'].split('_')[0]
        
        wer = jiwer.wer(
                text_to_decipher,
                text_on_fragment,
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )

        cer = jiwer.cer(
                text_to_decipher,
                text_on_fragment,
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )
        
        mer = jiwer.mer(
                text_to_decipher,
                text_on_fragment,
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )
        
        
        if label_type == "inpost":
            inpost_wer.append(wer)
        elif label_type == "dhl":
            dhl_wer.append(wer)
        elif label_type == "pocztex":
            pocztex_wer.append(wer)
        else:
            dpd_wer.append(wer)
        
        if class_int == 0:
            nadawca_scores.append(wer)
            nadawca_scores_cer.append(cer)
            nadawca_scores_mer.append(mer)
        elif class_int == 1:
            odbiorca_scores.append(wer)
            odbiorca_scores_cer.append(cer)
            odbiorca_scores_mer.append(mer)
        elif class_int == 2:
            uwagi_scores.append(wer)
            uwagi_scores_cer.append(cer)
            uwagi_scores_mer.append(mer)
        
        # cały fragment w dół jedynie dla wglądu w wyniki w razie potrzeby
        if k in [4, 15, 17]:
            
            # dla wglądu w to co model wykrywa (tekst) i porównania co powinno wykryć, odkomentować ponizsza linijke
            #####
            # print("Co wykryło: \n ######### \n", text_on_fragment, "\n #########, co miało być: ######### \n", text_to_decipher, "\n\n")
            #####
            
            # umieszczanie bounding boxów na obrazach
            d = pytesseract.image_to_data(img, output_type=Output.DICT)
            n_boxes = len(d['level'])
            for j in range(n_boxes):
                (x, y, w, h) = (d['left'][j], d['top'][j], d['width'][j], d['height'][j])
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.rectangle(fragment, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # dla zapisu przykładowych detekcji odkomentowac ponizsze dwie linijki
            #####
            # cv2.imwrite(f"tess_view_i{i}_k{k}.png", img)
            # cv2.imwrite(f"wycinek_i{i}_k{k}.png", fragment)
            ####
            
            
print("Miary dla detekcji zbioru treningowego, z podziałem na typ informacji: \n")
print("Uwagi WER:", np.mean(uwagi_scores))
print("Nadawca WER:", np.mean(nadawca_scores))
print("Odbiorca WER:", np.mean(odbiorca_scores))
print("############")
print("Uwagi CER:", np.mean(uwagi_scores_cer))
print("Nadawca CER:", np.mean(nadawca_scores_cer))
print("Odbiorca CER:", np.mean(odbiorca_scores_cer))
print("############")
print("Uwagi MER:", np.mean(uwagi_scores_mer))
print("Nadawca MER:", np.mean(nadawca_scores_mer))
print("Odbiorca MER:", np.mean(odbiorca_scores_mer))
print("\nMiary dla detekcji zbioru treningowego, z podziałem na typ przewoźnika:")
print("Inpost WER: ", np.mean(inpost_wer))
print("Pocztex WER: ", np.mean(pocztex_wer))
print("DPD WER: ", np.mean(dpd_wer))
print("DHL WER: ", np.mean(dhl_wer))

Miary dla detekcji zbioru treningowego, z podziałem na typ informacji: 

Uwagi WER: 0.8982683982683981
Nadawca WER: 0.8784149184149185
Odbiorca WER: 0.8906549659490834
############
Uwagi CER: 0.8982683982683981
Nadawca CER: 0.8784149184149185
Odbiorca CER: 0.8906549659490834
############
Uwagi MER: 0.6987322201607916
Nadawca MER: 0.8572416472416472
Odbiorca MER: 0.8215845592316179

Miary dla detekcji zbioru treningowego, z podziałem na typ przewoźnika:
Inpost WER:  0.8845777979924321
Pocztex WER:  0.8731746031746032
DPD WER:  0.8663517768780927
DHL WER:  0.9228888888888889


In [13]:
# teraz obliczenia dla zbioru walidacyjnego

class_names_dict = {0:"nadawca",
                    1:"odbiorca",
                    2:"uwagi"}

nadawca_scores = []
odbiorca_scores = []
uwagi_scores = []

nadawca_scores_cer = []
odbiorca_scores_cer = []
uwagi_scores_cer = []

nadawca_scores_mer = []
odbiorca_scores_mer = []
uwagi_scores_mer = []

inpost_wer = []
dhl_wer = []
dpd_wer = []
pocztex_wer = []


for k, result in enumerate(loaded_val_results):
    orig_image = result.orig_img
    fullpath = result.path
    number = int(fullpath.split("_")[-1][:-5]) # zadziała dla train i val, nie dla test
    detected_classes = result.boxes.cls
    detected_boxes = result.boxes
    
    for i, class_detected in enumerate(detected_classes):
        class_int = int(class_detected.numpy())
        [x1, y1, x2, y2] = detected_boxes[i].xyxy.numpy().astype(int)[0]
        fragment = orig_image[y1:y2, x1:x2, :] # działa! 
        
        
        img = get_grayscale(fragment)
        img = cv2.GaussianBlur(img,(5,5),0)

        img = thresholding(img)
        
        #open
        kernel = np.ones((4,4),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_OPEN, kernel)
        
        #close
        kernel = np.ones((4,4),np.uint8)
        img = cv2.morphologyEx(img, cv2.MORPH_CLOSE, kernel)
        

        text_on_fragment = pytesseract.image_to_string(img)

        text_category = class_names_dict[class_int]
        text_to_decipher = generated_df.iloc[number][text_category]
        label_type = generated_df.iloc[number]['label_type'].split('_')[0]
        
        wer = jiwer.wer(
                text_to_decipher,
                text_on_fragment,
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )

        cer = jiwer.cer(
                text_to_decipher,
                text_on_fragment,
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )
        
        mer = jiwer.mer(
                text_to_decipher,
                text_on_fragment,
                truth_transform=transforms,
                hypothesis_transform=transforms,
            )
        
        
        if label_type == "inpost":
            inpost_wer.append(wer)
        elif label_type == "dhl":
            dhl_wer.append(wer)
        elif label_type == "pocztex":
            pocztex_wer.append(wer)
        else:
            dpd_wer.append(wer)
        
        if class_int == 0:
            nadawca_scores.append(wer)
            nadawca_scores_cer.append(cer)
            nadawca_scores_mer.append(mer)
        elif class_int == 1:
            odbiorca_scores.append(wer)
            odbiorca_scores_cer.append(cer)
            odbiorca_scores_mer.append(mer)
        elif class_int == 2:
            uwagi_scores.append(wer)
            uwagi_scores_cer.append(cer)
            uwagi_scores_mer.append(mer)
            
        # cały fragment w dół jedynie dla wglądu w wyniki w razie potrzeby
        if k in [4, 15, 17]:
            
            # dla wglądu w to co model wykrywa (tekst) i porównania co powinno wykryć, odkomentować ponizsza linijke
            #####
            # print("Co wykryło: \n ######### \n", text_on_fragment, "\n #########, co miało być: ######### \n", text_to_decipher, "\n\n")
            #####
            
            # umieszczanie bounding boxów na obrazach
            d = pytesseract.image_to_data(img, output_type=Output.DICT)
            n_boxes = len(d['level'])
            for j in range(n_boxes):
                (x, y, w, h) = (d['left'][j], d['top'][j], d['width'][j], d['height'][j])
                cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
                cv2.rectangle(fragment, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            # dla zapisu przykładowych detekcji odkomentowac ponizsze dwie linijki
            #####
            # cv2.imwrite(f"tess_view_i{i}_k{k}.png", img)
            # cv2.imwrite(f"wycinek_i{i}_k{k}.png", fragment)
            ####
     
            
print("Miary dla detekcji zbioru WALIDACYJNEGO, z podziałem na typ informacji: \n")
print("Uwagi WER:", np.mean(uwagi_scores))
print("Nadawca WER:", np.mean(nadawca_scores))
print("Odbiorca WER:", np.mean(odbiorca_scores))
print("############")
print("Uwagi CER:", np.mean(uwagi_scores_cer))
print("Nadawca CER:", np.mean(nadawca_scores_cer))
print("Odbiorca CER:", np.mean(odbiorca_scores_cer))
print("############")
print("Uwagi MER:", np.mean(uwagi_scores_mer))
print("Nadawca MER:", np.mean(nadawca_scores_mer))
print("Odbiorca MER:", np.mean(odbiorca_scores_mer))
print("\nMiary dla detekcji zbioru WALIDACYJNEGO, z podziałem na typ przewoźnika:")
print("Inpost WER: ", np.mean(inpost_wer))
print("Pocztex WER: ", np.mean(pocztex_wer))
print("DPD WER: ", np.mean(dpd_wer))
print("DHL WER: ", np.mean(dhl_wer))

Miary dla detekcji zbioru WALIDACYJNEGO, z podziałem na typ informacji: 

Uwagi WER: 0.8973684210526316
Nadawca WER: 0.8730158730158731
Odbiorca WER: 0.9765647051361338
############
Uwagi CER: 0.8973684210526316
Nadawca CER: 0.8730158730158731
Odbiorca CER: 0.9765647051361338
############
Uwagi MER: 0.7472431077694235
Nadawca MER: 0.8239435168006598
Odbiorca MER: 0.7944896528930142

Miary dla detekcji zbioru WALIDACYJNEGO, z podziałem na typ przewoźnika:
Inpost WER:  0.8953848833159178
Pocztex WER:  0.8753086419753087
DPD WER:  1.2559567416710276
DHL WER:  0.7938271604938271
